# Data Science z Python 3.10. CNN w TensorFlow
## 🇬🇧 Data Science with Python 3.10. CNN with TensorFlow
#### 👨‍🏫 PhD Wojciech Oronowicz-Jaśkowiak
#### 🤖 https://www.oronowicz-jaśkowiak.pl/studenci/pjatk

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import keras
import plotly.graph_objects as go

from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications import ResNet50

In [ ]:
base_dir = '/media/linux/...'
raw_no_of_files = {}
classes = ['1', '2', '3']
for dir in classes:
    raw_no_of_files[dir] = len(os.listdir(os.path.join(base_dir, dir)))
raw_no_of_files.items()

In [ ]:
base_dir = '/media/linux/...'
data_dir = '/media/linux/...'

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

train_dir = os.path.join(data_dir, 'train')    # katalog zbioru treningowego
valid_dir = os.path.join(data_dir, 'valid')    # katalog zbioru walidacyjnego
test_dir = os.path.join(data_dir, 'test')      # katalog zbioru testowego

train_class1_dir = os.path.join(train_dir, '1')
train_class2_dir = os.path.join(train_dir, '2')
train_class3_dir = os.path.join(train_dir, '3')

valid_class1_dir = os.path.join(valid_dir, '1')
valid_class2_dir = os.path.join(valid_dir, '2')
valid_class3_dir = os.path.join(valid_dir, '3')

test_class1_dir = os.path.join(test_dir, '1')
test_class2_dir = os.path.join(test_dir, '2')
test_class3_dir = os.path.join(test_dir, '3')

for directory in (train_dir, valid_dir, test_dir):
    if not os.path.exists(directory):
        os.mkdir(directory)

dirs = [train_class1_dir, train_class2_dir, train_class3_dir,
        valid_class1_dir, valid_class2_dir, valid_class3_dir,
        test_class1_dir, test_class2_dir, test_class3_dir]

for dir in dirs:
    if not os.path.exists(dir):
        os.mkdir(dir)        

In [ ]:
class1_fnames = os.listdir(os.path.join(base_dir, '1'))
class2_fnames = os.listdir(os.path.join(base_dir, '2'))
class3_fnames = os.listdir(os.path.join(base_dir, '3'))

class1_fnames = [fname for fname in class1_fnames if fname.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]
class2_fnames = [fname for fname in class2_fnames if fname.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]
class3_fnames = [fname for fname in class3_fnames if fname.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]

In [ ]:
size = min(len(class1_fnames), len(class2_fnames), len(class3_fnames))

train_size = int(np.floor(0.7 * size))
valid_size = int(np.floor(0.2 * size))
test_size = size - train_size - valid_size

train_idx = train_size
valid_idx = train_size + valid_size
test_idx = train_size + valid_size + test_size

In [ ]:
for i, fname in enumerate(class1_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, '1', fname)
        dst = os.path.join(train_class1_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, '1', fname)
        dst = os.path.join(valid_class1_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, '1', fname)
        dst = os.path.join(test_class1_dir, fname)
        shutil.copyfile(src, dst)

for i, fname in enumerate(class2_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, '2', fname)
        dst = os.path.join(train_class2_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, '2', fname)
        dst = os.path.join(valid_class2_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, '2', fname)
        dst = os.path.join(test_class2_dir, fname)
        shutil.copyfile(src, dst) 

for i, fname in enumerate(class3fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, '3', fname)
        dst = os.path.join(train_class3_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, '3', fname)
        dst = os.path.join(valid_class3_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, '3', fname)
        dst = os.path.join(test_class3_dir, fname)
        shutil.copyfile(src, dst)        

print('1 - zbiór treningowy', len(os.listdir(train_class1_dir)))
print('1 - zbiór walidacyjny', len(os.listdir(valid_class1_dir)))
print('1 - zbiór testowy', len(os.listdir(test_class1_dir)))

print('2 - zbiór treningowy', len(os.listdir(train_class2_dir)))
print('2 - zbiór walidacyjny', len(os.listdir(valid_class2_dir)))
print('2 - zbiór testowy', len(os.listdir(test_class2_dir))) 

print('3 - zbiór treningowy', len(os.listdir(train_class3_dir)))
print('3 - zbiór walidacyjny', len(os.listdir(valid_class3_dir)))
print('3 - zbiór testowy', len(os.listdir(test_class3_dir))) 

### <a name='a4'></a> Augmentcja danych

In [ ]:
train_datagen = ImageDataGenerator()

valid_datagen = ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(directory=valid_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='categorical')

In [ ]:
batch_size = 20
steps_per_epoch = train_size // batch_size
validation_steps = valid_size // batch_size

In [ ]:
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
conv_base.trainable = True

def print_layers(model):
    for layer in model.layers:
        print(f'layer_name: {layer.name:13} trainable: {layer.trainable}')

print_layers(conv_base)

In [ ]:
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

print_layers(conv_base)

In [ ]:
model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(units=256, activation='relu'))
model.add(layers.Dense(units=3, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(train_generator, epochs=1, steps_per_epoch=2)

In [ ]:
model.save('saved_model/my_model2')

In [ ]:
new_model = keras.models.load_model('saved_model/my_model')
new_model.summary()

In [ ]:
#pip install tf2onnx
#python -m tf2onnx.convert --saved-model /home/linux/saved_model/my_model --output /home/linux/saved_model/my_model/tfmodel.onnx